In [ ]:
%load_ext autoreload
%reload_ext autoreload
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


# dataloading

> Dataloaders for the neural networks. Uses pytorch dataloaders and converts to jax format

In [ ]:
#| default_exp dataloading

In [ ]:
#| hide
import nbdev
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#| export

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import jax.numpy as jnp

import numpy as np

## Dummy Dataset

In [ ]:
#| export
# create a dummy dataset for testing
class DummyDataset(Dataset):
    """Dummy dataset for testing"""

    def __init__(
        self,
        n_samples=100,  # number of samples
        n_features=5,  # number of features
        n_targets=1,  # number of targets
        seed=42,  # random seed
    ):
        self.n_samples = n_samples
        self.n_features = n_features
        self.n_targets = n_targets
        self.seed = seed
        self.x = jnp.array(np.random.rand(n_samples, n_features))
        self.y = jnp.array(np.random.rand(n_samples, n_targets))

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [ ]:
#| export
def numpy_collate(batch):
    if isinstance(batch[0], np.ndarray):
        return np.stack(batch)
    elif isinstance(batch[0], (tuple, list)):
        transposed = zip(*batch)
        return [numpy_collate(samples) for samples in transposed]
    else:
        return np.array(batch)


class NumpyLoader(DataLoader):
    """A dataloader that uses numpy_collate
    to allow numpy arrays instead of torch tensors"""

    def __init__(
        self,
        dataset,
        batch_size=1,
        shuffle=False,
        sampler=None,
        batch_sampler=None,
        num_workers=0,
        pin_memory=False,
        drop_last=False,
        timeout=0,
        worker_init_fn=None,
    ):
        super(self.__class__, self).__init__(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            sampler=sampler,
            batch_sampler=batch_sampler,
            num_workers=num_workers,
            collate_fn=numpy_collate,
            pin_memory=pin_memory,
            drop_last=drop_last,
            timeout=timeout,
            worker_init_fn=worker_init_fn,
        )

In [ ]:
#| test
dummy_dataset = DummyDataset(n_samples=250, n_features=4, n_targets=4, seed=42)
dataloader = NumpyLoader(dummy_dataset, batch_size=32, shuffle=True)

for x, y in dataloader:
    print(x.shape, y.shape)
    break

test_eq(x.shape, (32, 4))
test_eq(y.shape, (32, 4))

(32, 4) (32, 4)


## Real Pendulum

In [ ]:
#| export
class RealPendulumDataset(Dataset):
    def __init__(
        self,
        root_path: str,  # path to data
        train_split: float,  # fraction of data to use for training
        train: bool = True,  # whether to use train or test data
    ):
        self.root_path = root_path
        self.headers = ["time", "theta", "theta_dot", "theta_ddot"]
        data = np.loadtxt(root_path + "extracted_angles.csv", delimiter=",")

        if train:
            data = data[: int(train_split * len(data))]
        else:
            data = data[int(train_split * len(data)) :]

        # separate x(theta, theta_dot) and y(theta_dot, theta_ddot)
        self.x = data[:, 1:3]
        self.y = data[:, 2:]

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return jnp.array(self.x[idx]), jnp.array(self.y[idx])

## Henon Heiles

In [ ]:
#| export
class HHDataset(Dataset):
    def __init__(
        self, root_path, train  # path to data  # whether to use train or test data
    ):
        if train:
            self.path = root_path + "train/train.csv"
        else:
            self.path = root_path + "valid/valid.csv"

        data = np.loadtxt(self.path, delimiter=",", skiprows=1)
        
        # make a dictionary with keys "x" and "y" with x being q1, q2, p1, p2 and y being dq1, dq2, dp1, dp2
        data = {"x": data[:, :4], "y": data[:, 5:]}
        self.x = data["x"][::400]
        self.y = data["y"][::400]

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return jnp.array(self.x[idx]), jnp.array(self.y[idx])

## Swinging Sticks

In [ ]:
#| export
class SwingingSticksDataset(Dataset):
    def __init__(
        self,
        root_path,  # path to data
        train_split,  # fraction of data to use for training
        train=True,  # whether to use train or test data
    ):
        self.root_path = root_path
        self.headers = [
            "time",
            "theta_a",
            "theta_A",
            "theta_a_dot",
            "theta_A_dot",
            "theta_a_ddot",
            "theta_A_ddot",
        ]
        data = np.loadtxt(root_path + "SSData.txt", delimiter="\t")

        if train:
            data = data[: int(train_split * len(data))]
        else:
            data = data[int(train_split * len(data)) :]

        # separate x and y
        self.x = data[:, 1:5]
        self.y = data[:, 3:]

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return jnp.array(self.x[idx]), jnp.array(self.y[idx])

In [ ]:
#| hide
nbdev.nbdev_export()